# Manually Serve the model 

In notebook '04-04-accident-recognition' we were able to use the retrained model to predict a 'severe' or 'moderate' car accident within an image.  

Next we will determine if we can manually connect with the model from the model server and model pipeline we have created and deployed.  This will be done through an API, which will be served directly from our container using Flask

In [3]:
!pip install ultralytics
from ultralytics import YOLO
from PIL import Image

from flask import request, Response, Flask  #create Flask web app and receive requests from frontent and send response back
import json  #convert the array of bounding boxes to JSON before returning it
import os

In [5]:
#location of your test file
my_image = 'images/carImage1.jpg'

#host route
my_route = 'http://localhost:5000'

#location of model in S3
model = YOLO("https://rhods-public.s3.amazonaws.com/demo-models/ic-models/accident/accident_detect.onnx", task="detect")

os.environ["MY_IMAGE"] = my_image
os.environ["MY_ROUTE"] = my_route

Found https://rhods-public.s3.amazonaws.com/demo-models/ic-models/accident/accident_detect.onnx locally at accident_detect.onnx


In [6]:
#check the status of our route
!curl "${MY_ROUTE}/status"

{"status":"ok"}


In [8]:
#call our prediction function and process our image through our deployed model
!(echo -n '{"image": "'; base64 "${MY_IMAGE}"; echo '"}') | curl -X POST -H "Content-Type: application/json" -d @- ${MY_ROUTE}/predictions

curl: Remote file name has no length!
curl: try 'curl --help' or 'curl --manual' for more information
<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>


In [ ]:
#define our predict function and pass the path to our model and car image.
def predict(best_model_path, car_image_path):
    model = YOLO(best_model_path)
    results = model.predict(car_image_path)
    return results[0]

In [ ]:
#In the photo draw boxes listing name, probability around each car (object type)
def draw_boxes_image(result):
    print("inside draw boxes image")
    #Image.fromarray(result.plot()[:,:,::-1])

In [ ]:
#call our predict function
prediction_results = predict("models/best.pt", "images/carImage1.jpg")
Image.fromarray(prediction_results.plot()[:,:,::-1])
#draw_boxes_image(prediction_results, image)